In [14]:
import keras
from keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import optimizers
import pandas as pd
import numpy as np
from keras.utils import to_categorical

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [15]:

img_width, img_height = 128, 128
TrainData = pd.read_csv('TrainAnnotations.csv')

Annotations = TrainData['annotation'].tolist()
#display(Image(FileNames[0]))
#TrainData.shape[0]

Annotations_one_hot = to_categorical(Annotations)

print(Annotations_one_hot)


FileNames = ['TrainData\\'+ fname for fname in TrainData['file_name'].tolist()]
Train_image = []

for i in range(TrainData.shape[0]):
    #img = resize(FileNames[i],(32,32,3))
    img = keras.preprocessing.image.load_img(FileNames[i],target_size = (128,128))
    img = image.img_to_array(img)
    img = img/255
    Train_image.append(img)

X = np.array(Train_image)

# Check if the images are RGB and change the channels likewise
if K.image_data_format() == 'channels_first':
  input_shape= (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)

Y = np.array(Annotations)



# create iterator
#it = datagen.flow(X, Y)


X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.2, random_state = 10 )
Y_train = np_utils.to_categorical(Y_train, 5)
Y_val = np_utils.to_categorical(Y_val, 5)





[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [16]:

def get_model(learning_rate = 0.0001):
  
  model = Sequential()
  model.add(Conv2D(32,(3,3), input_shape = input_shape)) #32 #(3,3)
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))

  model.add(Conv2D(64,(3,3))) #64
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))

  model.add(Conv2D(128,(2,2))) #128
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))

  

  

  model.add(Flatten())

  model.add(Dense(128))#64
  model.add(Activation('relu'))#relu
  model.add(Dropout(0.5))#0.5
  model.add(Dense(64))#64
  model.add(Activation('relu'))#relu
  model.add(Dropout(0.5))#0.5
  model.add(Dense(5))
  model.add(Activation('softmax'))

  optimizer = optimizers.adam(learning_rate)
                                              
  model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
  
  return model

In [17]:
batch_sizes = [16]

learning_rates = [0.001]


histories = []

epochs = 100

i = 0

for batch_size in batch_sizes:
  
  for learning_rate in learning_rates:
    
    print("Batch Size : {} Learning rate: {}".format(batch_size, learning_rate))
    
    #train_datagen = ImageDataGenerator(rescale = 1./255)

   # train_generator = train_datagen.flow_from_directory('TrainData\\', target_size =(img_width, img_height),class_mode = 'categorical')

    #validation_generator = train_datagen.flow_from_directory( val_data_dir, target_size =(img_width, img_height), batch_size = batch_size, class_mode ='categorical') 
 # history = model.fit_generator(train_generator, steps_per_epoch = num_train // batch_size, epochs = epochs, validation_data = validation_generator, validation_steps = num_val// batch_size)
    
    model = get_model(learning_rate)

  

    Train_fit = model.fit(X_train,Y_train,batch_size = 16, epochs = 100, validation_data= (X_val, Y_val))#16#50
    
    
    # fits the model on batches with real-time data augmentation:
    #Train_fit = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),epochs=50,validation_data= (X_val, Y_val))
    
    
    
    #model.save_weights("model_new"+ str(i+1) +".h5")
    
    model.save("model_new"+ str(i+1) +".h5")
    
    histories.append(model.history.history)
    
    i = i + 1
    

    



Batch Size : 16 Learning rate: 0.001
Train on 1020 samples, validate on 255 samples
Epoch 1/100
1020/1020 [==============================] - 5s 5ms/step - loss: 1.5542 - accuracy: 0.3363 - val_loss: 1.5333 - val_accuracy: 0.3412
Epoch 2/100
1020/1020 [==============================] - 5s 4ms/step - loss: 1.4356 - accuracy: 0.3882 - val_loss: 1.3759 - val_accuracy: 0.4980
Epoch 3/100
1020/1020 [==============================] - 5s 4ms/step - loss: 1.3327 - accuracy: 0.4804 - val_loss: 1.1849 - val_accuracy: 0.5216
Epoch 4/100
1020/1020 [==============================] - 5s 5ms/step - loss: 1.2093 - accuracy: 0.5206 - val_loss: 1.0657 - val_accuracy: 0.6039
Epoch 5/100
1020/1020 [==============================] - 5s 5ms/step - loss: 1.0540 - accuracy: 0.5814 - val_loss: 1.0029 - val_accuracy: 0.5922
Epoch 6/100
1020/1020 [==============================] - 5s 5ms/step - loss: 0.9087 - accuracy: 0.6255 - val_loss: 0.8860 - val_accuracy: 0.5765
Epoch 7/100
1020/1020 [=======================

1020/1020 [==============================] - 5s 5ms/step - loss: 0.0783 - accuracy: 0.9735 - val_loss: 1.4824 - val_accuracy: 0.8235
Epoch 57/100
1020/1020 [==============================] - 5s 4ms/step - loss: 0.0907 - accuracy: 0.9627 - val_loss: 1.0030 - val_accuracy: 0.8392
Epoch 58/100
1020/1020 [==============================] - 5s 4ms/step - loss: 0.0801 - accuracy: 0.9755 - val_loss: 1.0286 - val_accuracy: 0.8353
Epoch 59/100
1020/1020 [==============================] - 5s 4ms/step - loss: 0.0760 - accuracy: 0.9696 - val_loss: 1.2666 - val_accuracy: 0.8275
Epoch 60/100
1020/1020 [==============================] - 5s 4ms/step - loss: 0.0651 - accuracy: 0.9725 - val_loss: 1.3630 - val_accuracy: 0.8078
Epoch 61/100
1020/1020 [==============================] - 5s 4ms/step - loss: 0.1020 - accuracy: 0.9559 - val_loss: 1.3334 - val_accuracy: 0.8157
Epoch 62/100
1020/1020 [==============================] - 5s 4ms/step - loss: 0.0837 - accuracy: 0.9716 - val_loss: 1.4176 - val_accuracy

In [18]:
import numpy
import sklearn.metrics as metrics
import glob
img_width, img_height = 128, 128
batch_size = 8
epochs = 100
Testpath = "TestData/*.jpg"
Testfiles = glob.glob(Testpath)
Test_image = []
for i in range (len(Testfiles)):
    img = keras.preprocessing.image.load_img(Testfiles[i],target_size = (128, 128))
    img = image.img_to_array(img)
    img = img/255
    Test_image.append(img)
   
   
X_test = np.array(Test_image)
YPredict_test = model.predict(X_test)
Predict_test = np.argmax(YPredict_test, axis=1)
Predict_one_hot = to_categorical(Predict_test)

print(Predict_one_hot)
set(Predict_test)
result = pd.DataFrame(Predict_one_hot)
result.to_csv("predict.csv",header=False, index=False)
'''
test_set_dir = "/content/drive/My Drive/NN-ProjectC/Project_C1/Test/"

num_test = len(os.listdir(test_set_dir))

print ("Number of images in test set: ", num_test)

model = get_model(0.0001)

model.load_weights("/content/model_new1.h5")
  
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_set_dir, target_size =(img_width, img_height), 
                                                  batch_size = batch_size, class_mode =None, shuffle = False) 

test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict_generator(test_generator, steps = test_steps_per_epoch)

print("PREDICTIONS: --->")
print(predictions)

predicted_classes = numpy.argmax(predictions, axis=1)

print("PREDICTED CLASSES: --->")
print (predicted_classes)
'''

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0.

'\ntest_set_dir = "/content/drive/My Drive/NN-ProjectC/Project_C1/Test/"\n\nnum_test = len(os.listdir(test_set_dir))\n\nprint ("Number of images in test set: ", num_test)\n\nmodel = get_model(0.0001)\n\nmodel.load_weights("/content/model_new1.h5")\n  \ntest_datagen = ImageDataGenerator(rescale=1./255)\n\ntest_generator = test_datagen.flow_from_directory(test_set_dir, target_size =(img_width, img_height), \n                                                  batch_size = batch_size, class_mode =None, shuffle = False) \n\ntest_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)\n\npredictions = model.predict_generator(test_generator, steps = test_steps_per_epoch)\n\nprint("PREDICTIONS: --->")\nprint(predictions)\n\npredicted_classes = numpy.argmax(predictions, axis=1)\n\nprint("PREDICTED CLASSES: --->")\nprint (predicted_classes)\n'